# multidms

## Overview of model

The `multidms` model applies to a case where you have DMS datasets for two or more homologs and are interested in identifying shifts in mutational effects between homologs.
To do so, the model defines one homolog as a reference homolog.
For each mutation, the model fits one parameter that quantifies the effect of the mutation in the reference homolog.
For each non-reference homolog, it also fits a shift parameter that quantifies the shift in the mutation's effect in the non-reference homolog relative to the reference.
Shift parameters can be regularized, encouraging most of them to be close to zero.
This regularization step is a useful way to eliminate the effects of experimental noise, and is most useful in cases where you expect most mutations to have the same effects between homologs, such as for homologs that are close relatives.

The model uses a global-epistasis function to disentangle the effects of multiple mutations on the same variant.
To do so, it assumes that mutational effects additively influence a latent biophysical property the protein (e.g., $\Delta G$ of folding).
The mutational-effect parameters described above operate at this latent level.
The global-epistasis function then assumes a sigmoidal relationship between a protein's latent property and its functional score measured in the experiment (e.g., log enrichment score).
Ultimately, mutational parameters, as well as ones controlling the shape of the sigmoid, are all jointly fit to maximize agreement between predicted and observed functional scores acorss all variants of all homologs.

## Detailed description of the model

For each variant $v$ from homolog $h$, we use a global-epistasis function $g$ to convert a latent phenotype $\phi$ to a functional score $f$:

$$f(v,h) = g_{\alpha}(\phi(v,h))$$

where $g$ is a sigmoid and $\alpha$ is a set of parameters encoding the sigmoid.

The latent phenotype is computed in the following way:

$$\phi(v,h) = c + \sum_{m \in v} (x_m + s_{m,h})$$

where:
* $c$ is the wildtype latent phenotype for the reference homolog.
* $x_m$ is the latent phenotypic effect of mutation $m$. See details below.
* $s_{m,h}$ is the shift of the effect of mutation $m$ in homolog $h$. These parameters are fixed to zero for the reference homolog. For non-reference homologs, they are defined in the same way as $x_m$ parameters.
* $v$ is the set of all mutations relative to the reference wildtype sequence (including all mutations that separate homolog $h$ from the reference homolog).

The $x_m$ variable is defined such that mutations are always relative to the reference homolog.
For example, if the wildtype amino acid at site 30 is an A in the reference homolog, and a G in a non-reference homolog, then a Y30G mutation in the non-reference homolog is recorded as an A30G mutation relative to the reference.
This way, each homolog informs the exact same parameters, even at sites that differ in wildtype amino acid.
These are encoded in a `BinaryMap` object, where all sites that are non-identical to the reference are 1's.

Ultimately, we fit parameters using a loss function with one term that scores differences between predicted and observed values and another that uses L1 regularization to penalize non-zero $s_{m,h}$ values:

$$ L_{\text{total}} = \sum_{h} \left[\sum_{v} L_{\text{fit}}(y_{v,h}, f(v,h)) + \lambda \sum_{m} |s_{m,h}|\right]$$

where:
* $L_{\text{total}}$ is the total loss function.
* $L_{\text{fit}}$ is a loss function that penalizes differences in predicted vs. observed functional scores.
* $y_{v,h}$ is the experimentally measured functional score of variant $v$ from homolog $h$.

## Model using matrix algebra

We compute a vector or predicted latent phenotypes $P_{h}$ as:

$$P_{h} = c + (X_h \cdot (β + S_h))$$

where:
* $β$ is a vector of all $β_m$ values.
* $S_h$ is a matrix of all $s_{m,h}$ values.
* $X_h$ is a sparse matrix, where rows are variants, columns are mutations (all defined relative to the reference homolog), and values are weights of 0's and 1's. These weights are used to compute the phenotype of each variant given the mutations present.
* $c$ is the same as above.

In the matrix algebra, the sum of $β_m$ and $S_{m,h}$ gives a vector of mutational effects, with one entry per mutation.
Multiplying the matrix $X_h$ by this vector gives a new vector with one entry per variant, where values are the sum of mutational effects, weighted by the variant-specific weights in $X_h$.
Adding the $c$ value to this vector will give a vector of predicted latent phenotypes for each variant.

Next, the global-epistasis function can be used to convert a vector of predicted latent phenotypes to a vector of predicted functional scores.

$$F_{h,pred} = g_{\alpha}(P_h)$$

Finally, this vector could be fed into a loss function and compared with a vector of observed functional scores.

## Import `Python` modules

In [1]:
import pandas as pd
import numpy as np
import re
import json
import binarymap as bmap
import matplotlib.pyplot as plt
import seaborn as sns
import jax
jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp
from jax.experimental import sparse
import jaxopt
import numpy as onp
from scipy.stats import pearsonr

## Strategy for converting mutations to be relative to the reference homolog

As described above, our strategy involves converting all mutations in all homologs to be relative to the amino-acid sequence of the reference homolog. The below function performs this step.

Note 1: we will probably need some special-purpose code to handle gaps. This isn't done yet.

In [2]:
def create_homolog_modeling_data(
    func_score_df:pd.DataFrame, 
    homologs:dict,
    homolog_name_col: str,
    substitution_col: str,
    func_score_col: str
):
    """
    Takes a dataframe for making a `BinaryMap` object, and adds
    a column where each entry is a list of mutations in a variant
    relative to the amino-acid sequence of the reference homolog.
    
    Parameters
    ----------

    func_score_df : pandas.DataFrame
        This should be in the same format as described in BinaryMap.
        
    homologs : dict
        A dictionary containing all possible target homolog 
        names (keys) and sequences (values).
    
    homolog_name_col : str
        The name of the column in func_score_df that identifies the
        homolog for a given variant. We require that the
        reference homolog variants are labeled as 'reference'
        in this column.
    
    substitution_col : str 
        The name of the column in func_score_df that
        lists mutations in each variant relative to the wildtype
        amino-acid sequence of the homolog in which they occur.
        
    func_score_col : str
        Column in func_scores_df giving functional score for each variant.
    
    Returns
    -------
        
    pd.DataFrame :
        A dataframe with a new column 'var_wrt_ref' that contains
        all variant mutations with respect to the wildtype.
    
    """
    
    def mutations_wrt_ref(mutations, hom_wtseq):
        """
        Takes a list of mutations for a given variant relative
        to its background homolog and returns a list of all
        mutations that separate the variant from the reference
        homolog.
        """
        
        # Compute the full amino-acid sequence of the
        # given variant
        mutated_homolog = list(hom_wtseq)
        for mutation in mutations.split():

            # TODO: Do we need to change the regex to allow
            # for gap '-' and stop '*' characters?
            pattern = r'(?P<aawt>\w)(?P<site>\d+)(?P<aamut>[\w\*])'
            match = re.search(pattern, mutation)
            assert match != None, mutation
            aawt = match.group('aawt')
            site = match.group('site')
            aamut = match.group('aamut')
            mutated_homolog[int(site)-1] = aamut
            
        hom_var_seq = ''.join(mutated_homolog)
        
        # Make a list of all mutations that separate the variant
        # from the reference homolog
        ref_muts = [
            f"{aaref}{i+1}{aavar}" 
            for i, (aaref, aavar) in enumerate(zip(homologs["reference"], hom_var_seq))
            if aaref != aavar
        ]
        
        return " ".join(ref_muts)

    # Duplicate the substitutions_col, then convert the respective functional scores
    func_score_df = func_score_df.assign(var_wrt_ref = func_score_df[substitution_col].values)
    for hom_name, hom_seq in homologs.items():
        if hom_name == "reference": continue
        hom_df = func_score_df.query(f"{homolog_name_col} == '{hom_name}'")
        hom_var_wrt_ref = [
            mutations_wrt_ref(muts, homologs[hom_name]) 
            for muts in hom_df[substitution_col]
        ]
        func_score_df.loc[hom_df.index.values, "var_wrt_ref"] = hom_var_wrt_ref   
    
    # Get list of all allowed substitutions that we will tune beta parameters for
    allowed_subs = {
        s for subs in func_score_df.var_wrt_ref
        for s in subs.split()
    }
    
    # Make BinaryMap representations for each homolog
    X, y = {}, {}
    for homolog, homolog_func_score_df in func_score_df.groupby("homolog"):
        ref_bmap = bmap.BinaryMap(
            homolog_func_score_df,
            substitutions_col="var_wrt_ref",
            allowed_subs=allowed_subs,
            func_score_col = None
        )
        
        # convert binarymaps into sparse arrays for model input
        X[homolog] = sparse.BCOO.fromdense(ref_bmap.binary_variants.toarray())
        
        # create jax array for functional score targets
        y[homolog] = jnp.array(func_score_df[func_score_col].values)
    
    return (X, y), func_score_df


Next, we will test the above function with a small test case. Below, we define variants from two imaginary homologs: "reference" and "2".

In [3]:
homologs = {
    "reference" : "MG",
    "2" : "MP"
}

In [4]:
test_dict = {
    'homolog' : ["reference","reference","reference","2","2","2","2","2"],
    'variant' : ['M1E', 'G2R', 'G2P', 'M1E', 'P2R', 'P2G', 'M1E P2G', 'M1E P2R'],
    'log2E' : [2, -7, -0.5, 2.3, -5, 0.4, 2.7, -2.7],
}
test_df = pd.DataFrame(test_dict)
test_df

,homolog,variant,log2E
0,reference,M1E,2.0
1,reference,G2R,-7.0
2,reference,G2P,-0.5
3,2,M1E,2.3
4,2,P2R,-5.0
5,2,P2G,0.4
6,2,M1E P2G,2.7
7,2,M1E P2R,-2.7


In [5]:
(X, y), updated_func_score_df = create_homolog_modeling_data(test_df, homologs, "homolog", "variant", "log2E")

print(f"Modeling Input Data: {X}")
print(f"Modeling Target Data: {y}")

updated_func_score_df

Modeling Input Data: {'2': BCOO(int8[5, 3], nse=6), 'reference': BCOO(int8[3, 3], nse=3)}
Modeling Target Data: {'2': DeviceArray([ 2. , -7. , -0.5,  2.3, -5. ,  0.4,  2.7, -2.7], dtype=float64), 'reference': DeviceArray([ 2. , -7. , -0.5,  2.3, -5. ,  0.4,  2.7, -2.7], dtype=float64)}


,homolog,variant,log2E,var_wrt_ref
0,reference,M1E,2.0,M1E
1,reference,G2R,-7.0,G2R
2,reference,G2P,-0.5,G2P
3,2,M1E,2.3,M1E G2P
4,2,P2R,-5.0,G2R
5,2,P2G,0.4,
6,2,M1E P2G,2.7,M1E
7,2,M1E P2R,-2.7,M1E G2R


## Implementing the full model with JAX

Below, we implement the $\phi$ and $g$ functions from above.

Next, we initialize values of free parameters for downstream optimization by JAX.

In [6]:
def initialize_model_params(homologs: dict, n_beta_shift_params: int, n_perceptron_units=1):
    """
    initialize a set of starting parameters for the JAX model.
    
    Parameters
    ----------
    
    homologs : dict
        A dictionary containing all possible target homolog 
        names (keys) and sequences (values).
    
    n_beta_shift_params: int
        The number of beta and shift parameters 
        (for each homolog) to initialize.
        
    n_perceptron_units : int
        The number of connections that are tuned
        for the shape of global epistasis before
        being fed into a sigmoid.
    
    Returns
    -------
    
    dict :
        all relevant parameters to be tuned with the JAX model.
    """
    
    params = {}
    seed = 0
    key = jax.random.PRNGKey(seed)

    # initialize beta parameters from normal distribution.
    params["β"] = jax.random.normal(shape=(n_beta_shift_params,), key=key)

    # initialize shift parameters
    for homolog in homologs.keys():
        # We expect most shift parameters to be close to zero
        params[f"S_{homolog}"] = jnp.zeros(shape=(n_beta_shift_params,))

    # Single bias param
    params["C_ref"] = jnp.zeros(shape=(1, ))
    
    # 'stretch' in x direction
    # 'shift' in the x direction
    # 'stretch' in the y direction
    # 'shift' in the y direction
    key, *subkeys = jax.random.split(key, num=5)
    params["α"]=dict(
        sig_stretch_x = jax.random.normal(shape=(n_perceptron_units,), key=subkeys[0]),
        sig_shift_x = jax.random.normal(shape=(1,), key=subkeys[1]),
        sig_stretch_y = jax.random.normal(shape=(n_perceptron_units,), key=subkeys[2]),
        sig_shift_y = jax.random.normal(shape=(1,), key=subkeys[3]),
    )

    return params

In [7]:
# number of columns in the sparse arrays are the number of beta/shift params to fit
params = initialize_model_params(homologs, n_beta_shift_params = X["reference"].shape[1])
params

{'β': DeviceArray([ 0.18784401, -1.28334229,  0.6494182 ], dtype=float64),
 'S_reference': DeviceArray([0., 0., 0.], dtype=float64),
 'S_2': DeviceArray([0., 0., 0.], dtype=float64),
 'C_ref': DeviceArray([0.], dtype=float64),
 'α': {'sig_stretch_x': DeviceArray([-1.31768732], dtype=float64),
  'sig_shift_x': DeviceArray([0.44991238], dtype=float64),
  'sig_stretch_y': DeviceArray([0.47178911], dtype=float64),
  'sig_shift_y': DeviceArray([-0.94836575], dtype=float64)}}

Define an objective function to optimize

In [8]:
# TODO centering wildtype -
# TODO static args for reference -
# TODO regularization -

@jax.jit
def ϕ(params:dict, X_h:jnp.array):
    """Model for predicting latent space"""
    
    return (X_h @ (params["β"] + params[f"S"])) + params["C_ref"]


@jax.jit
def g(α:dict, z_h:jnp.array):
    """Model for global epistasis as 'flexible' sigmoid."""
     
    activations = jax.nn.sigmoid(α["sig_stretch_x"] * z_h[:, None] + α["sig_shift_x"])
#     wt_activations = jax.nn.sigmoid(α["sig_stretch_x"] * α["C_ref"] + α["sig_shift_x"])
#     activations -= wt_activations
    return (α["sig_stretch_y"] * activations) + α["sig_shift_y"] # sig shift y needs to be 0 for the reference


@jax.jit
def cost_smooth(params, data, δ=1):
    """Cost (Objective) function as a sum of huber loss across all homologs"""

    X, y = data
    loss = 0   
    
    # Sum the huber loss across all homologs
    for homolog, X_h in X.items():
        
        # Fix the shift parameters for reference to 0
        # Static arguments for reference params?
        S_h = jnp.where(
            homolog=="reference", 
            jnp.zeros(len(params['β'])), 
            params[f"S_{homolog}"]
        )
        
        # Subset the params being passed into latent prediction, ϕ
        h_params = {"β":params["β"], "S":S_h, "C_ref":params["C_ref"]}
        
        z_h = ϕ(h_params, X_h)
        
        # Pass the latent predictions through GE model prediction
        # all GE specific parameters are stored in α
        y_h_predicted = g(params["α"], z_h)
        
        # compute loss at current parameter state.
        loss += jaxopt.loss.huber_loss(y[homolog], y_h_predicted, δ).mean()

    return loss

Calculate initial loss with random parameters

In [9]:
cost_smooth(params, (X, y))

DeviceArray(4.82309878, dtype=float64)

In [10]:
tol = 1e-6
maxiter = 1000
solver = jaxopt.GradientDescent(cost_smooth, tol=tol, maxiter=maxiter)

Optimize the entire model at once with gradient decent

In [11]:
params, state = solver.run(params, data=(X, y))
params

{'C_ref': DeviceArray([-0.41645006], dtype=float64),
 'S_2': DeviceArray([-0.08995514,  0.06204292, -0.22570208], dtype=float64),
 'S_reference': DeviceArray([0., 0., 0.], dtype=float64),
 'α': {'sig_shift_x': DeviceArray([0.79438156], dtype=float64),
  'sig_shift_y': DeviceArray([-0.05026768], dtype=float64),
  'sig_stretch_x': DeviceArray([-1.02919693], dtype=float64),
  'sig_stretch_y': DeviceArray([0.00032971], dtype=float64)},
 'β': DeviceArray([ 0.02419704, -1.12096377,  0.19726752], dtype=float64)}

## Simulation Fit

* TODO J: Try removing for loop in cost function (only compute cost on homolog)

* TODO H: Get will's notebook running in our environment (positive control for env)

* TODO J: jnp where culprit:
    * Try removing the shift parameters & update Phi accordingly
    * static args for reference

* TODO H: Run the Ab-CGGnaive_DMS with our model
* TODO H: Smaller simulated sequence

In [12]:
simulated_dataset = pd.read_csv("results/simulated_dataset_v1.csv")
simulated_dataset_lib1 = simulated_dataset.query("library == 'lib_1'").copy()
simulated_dataset_lib1.aa_substitutions.fillna("", inplace=True)
simulated_dataset_lib1 = simulated_dataset_lib1.sample(n=15000)

In [13]:
homologs = json.load(open("results/homolog_aa_seqs.json", "r"))
homologs["reference"] = homologs['1']
homologs["H2"] = homologs['2']
del homologs['1']
del homologs['2']
homologs

{'reference': 'RSVILRAYTNSRVKRVILCNNDLPIRNIRLMMILHNSDASFSTPVGLRSG',
 'H2': 'RVVILRAYTNSRVKRIKLCNNDRPIRNIRTMMIEHNSDAKFHTPYGLDSG'}

In [14]:
(X, y), updated_sim_func_score_df = create_homolog_modeling_data(
                            simulated_dataset_lib1, 
                            homologs, 
                            "homolog", 
                            "aa_substitutions",
                            "observed_enrichment"
                        )
print(f"Modeling Input Data: {X}")
print(f"Modeling Target Data: {y}")

updated_sim_func_score_df

Modeling Input Data: {'H2': BCOO(int8[7533, 1000], nse=86632), 'reference': BCOO(int8[7467, 1000], nse=14200)}
Modeling Target Data: {'H2': DeviceArray([0.97637767, 1.        , 1.00481545, ..., 0.06785799,
             1.        , 0.00103619], dtype=float64), 'reference': DeviceArray([0.97637767, 1.        , 1.00481545, ..., 0.06785799,
             1.        , 0.00103619], dtype=float64)}


,library,barcode,variant_call_support,codon_substitutions,aa_substitutions,n_codon_substitutions,n_aa_substitutions,latent_phenotype,observed_phenotype,observed_enrichment,homolog,var_wrt_ref
13743,lib_1,GATATCGCTACTGACA,1,CGG12TTG,R12L,1,1,3.469912,-0.034489,0.976378,reference,R12L
64881,lib_1,GCGAGGGGAGAAGTGG,1,NaN,,0,0,5.612939,0.000000,1.000000,H2,S2V V16I I17K L23R L30T L34E S40K S42H V45Y R48D
9091,lib_1,CCTGAACACTTAGCCT,1,ATC28CAA CAC35ATC,I28Q H35I,2,2,6.261448,0.006931,1.004815,reference,I28Q H35I
59455,lib_1,CGACTACCAAATAATA,3,CGT15GAA,R15E,1,1,5.614693,0.000009,1.000006,H2,S2V R15E V16I I17K L23R L30T L34E S40K S42H V4...
72388,lib_1,TGCCCATTGTTACCAC,1,CGT15GCA AAG17TTT,R15A K17F,2,2,-1.917020,-2.948603,0.129533,H2,S2V R15A V16I I17F L23R L30T L34E S40K S42H V4...
...,...,...,...,...,...,...,...,...,...,...,...,...
13769,lib_1,GATCACTTGGATACTG,1,TGC19ATG AAC21ACA AGT40CAC,C19M N21T S40H,3,3,-2.593791,-3.817305,0.070938,reference,C19M N21T S40H
64131,lib_1,GCAAGGGAACCCATGC,3,AAA14GTA ATG31CAG,K14V M31Q,2,2,4.753101,-0.007126,0.995073,H2,S2V K14V V16I I17K L23R L30T M31Q L34E S40K S4...
61828,lib_1,CTGCGCTTTAGTGTTC,3,AGA1GCC TAC8CAG,R1A Y8Q,2,2,-2.638819,-3.881337,0.067858,H2,R1A S2V Y8Q V16I I17K L23R L30T L34E S40K S42H...
57316,lib_1,CAGTGCTTCGTAGCCA,2,NaN,,0,0,5.612939,0.000000,1.000000,H2,S2V V16I I17K L23R L30T L34E S40K S42H V45Y R48D


In [15]:
# number of columns in the sparse arrays are the number of beta/shift params to fit
params = initialize_model_params(homologs, n_beta_shift_params = X["reference"].shape[1])

In [16]:
for key, value in params.items():
    if key == "α":
        for key_a, value_a in value.items():
            print(f"Parameter {key_a} has shape: {value_a.shape}")        
    else:
        print(f"Parameter {key} has shape: {value.shape}")

Parameter β has shape: (1000,)
Parameter S_reference has shape: (1000,)
Parameter S_H2 has shape: (1000,)
Parameter C_ref has shape: (1,)
Parameter sig_stretch_x has shape: (1,)
Parameter sig_shift_x has shape: (1,)
Parameter sig_stretch_y has shape: (1,)
Parameter sig_shift_y has shape: (1,)


In [17]:
cost_smooth(params, (X, y))

DeviceArray(1.24799972, dtype=float64)

In [18]:
tol = 1e-6
maxiter = 10
solver = jaxopt.GradientDescent(cost_smooth, tol=tol, maxiter=maxiter)

In [19]:
params, state = solver.run(params, data=(X, y))

In [20]:
cost_smooth(params, (X, y))

DeviceArray(0.20697753, dtype=float64)

## Empirical Data (TODO)